<h1>Проект для «Викишоп» с использованием BERT<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-описание-данных" data-toc-modified-id="Загрузка-и-описание-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и описание данных</a></span><ul class="toc-item"><li><span><a href="#Вывод-по-загрузке-данных" data-toc-modified-id="Вывод-по-загрузке-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод по загрузке данных</a></span></li></ul></li><li><span><a href="#Эмбеддинги-BERT" data-toc-modified-id="Эмбеддинги-BERT-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Эмбеддинги BERT</a></span></li><li><span><a href="#Подготовка-данных-и-обучение-моделей" data-toc-modified-id="Подготовка-данных-и-обучение-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка данных и обучение моделей</a></span><ul class="toc-item"><li><span><a href="#Предсказание-лучшей-модели" data-toc-modified-id="Предсказание-лучшей-модели-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Предсказание лучшей модели</a></span></li></ul></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Описание проекта

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 
Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.
Постройте модель со значением метрики качества F1 не меньше 0.75. 

# Инструкция для проекта

Решить задачу можно как с помощью BERT, так и без этой нейронки. Если хотите попробовать BERT —
Выполните проект локально. В тренажере тетрадь Jupyter ограничена 4 ГБ оперативной памяти — для проекта с BERT этого может не хватить.

Упомяните BERT в заголовке проекта в первой ячейке:

Выполнить проект без BERT можно локально или в нашем тренажёре.
В любом случае алгоритм решения выглядит так:
Загрузите и подготовьте данные.
Обучите разные модели.
Сделайте выводы.

In [32]:
import pandas as pd
import numpy as np
import time
import nltk
import torch
import transformers
import warnings
warnings.simplefilter("ignore")

from tqdm import notebook
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression

from tqdm.notebook import tqdm
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from catboost import CatBoostClassifier

In [2]:
RANDOM_STATE = 42

### Загрузка и описание данных

Данные находятся в файле `toxic_comments.csv`

Текст комментария содержится в столбце **text**

Целевой признак - **toxic**

In [3]:
file_local = '/Users/alexfil/Desktop/Practicum/Проекты/ml_texts/toxic_comments.csv'
file_ya = 'https://code.s3.yandex.net/datasets/toxic_comments.csv'

In [4]:
try:
    df = pd.read_csv(file_local) 
except:
    df = pd.read_csv(file_ya)

In [5]:
df.head(5)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [6]:
df.shape

(159292, 3)

Удалим неизвестный столбец:

In [7]:
df = df.drop('Unnamed: 0', axis = 1)
df.head(5)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [8]:
df.shape

(159292, 2)

In [9]:
df.isna().sum()

text     0
toxic    0
dtype: int64

In [10]:
df.duplicated().sum()

0

#### Вывод по загрузке данных

1. Загружен датасет размерностью 159292 строки и 3 столбца
2. Произведено удаление неизвестного столбца 'Unnamed: 0'
3. Пропущенных значений и дубликатов не выявленно
4. Работать будем с итоговым датасетом размерностью 159292 строки и 2 столбца

### Эмбеддинги BERT

Чтобы не создавать эмбеддинги слишком долго, из выборки используется только 500 случайных элементов:

In [11]:
df = df.sample(n=500, random_state=RANDOM_STATE)

In [12]:
df['toxic'].value_counts()

toxic
0    444
1     56
Name: count, dtype: int64

Инициализируем токенизатор как объект класса BertTokenizer(). Передадим ему аргумент vocab_file — это файл со словарём, на котором обучалась модель:

In [13]:
vocab_file = '/Users/alexfil/Desktop/Practicum/Теория/ML texts/Bert/sentence_ru_cased_L-12_H-768_A-12_pt_v1/vocab.txt'

In [14]:
tokenizer = transformers.BertTokenizer(vocab_file=vocab_file)

Преобразуем текст в номера токенов из словаря методом encode() с добавлением токенов начала и конца текста

In [15]:
tokenized = df['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True))

Определение максимальной длины токена

In [16]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

print('Максимальная длина токена:', max_len) 

Максимальная длина токена: 1542


Применим метод padding (англ. «отступ»), чтобы после токенизации длины исходных текстов в корпусе были равными. Только при таком условии будет работать модель BERT. 
Также уменьшим размер токенов до 512

In [17]:
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
padded = padded[:, :512] 

Теперь поясним модели, что нули не несут значимой информации. Это нужно для компоненты модели, которая называется «внимание» (англ. attention). Отбросим эти токены и «создадим маску» для действительно важных токенов, то есть укажем нулевые и не нулевые значения:

In [18]:
attention_mask = np.where(padded != 0, 1, 0)

Инициализируем саму модель класса BertModel. Передадим ей файл с предобученной моделью и конфигурацией: 

In [19]:
config = transformers.BertConfig.from_json_file(
    '/Users/alexfil/Desktop/Practicum/Теория/ML texts/Bert/sentence_ru_cased_L-12_H-768_A-12_pt_v1/config.json'
)
model = transformers.BertModel.from_pretrained(
    '/Users/alexfil/Desktop/Practicum/Теория/ML texts/Bert/sentence_ru_cased_L-12_H-768_A-12_pt_v1/pytorch_model.bin',
    config=config
)

Преобразование текстов в эмбеддинги:

In [20]:
batch_size = 1
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(
            padded[batch_size * i:batch_size *(i + 1)]
        ) 
        attention_mask_batch = torch.LongTensor(
            attention_mask[batch_size * i:batch_size * (i + 1)]
        )
        with torch.no_grad():
            batch_embeddings = model(
                batch, attention_mask=attention_mask_batch
            )
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/500 [00:00<?, ?it/s]

Соберём все эмбеддинги в матрицу признаков вызовом функции concatenate():

In [21]:
X = np.concatenate(embeddings)
y = df['toxic']

In [22]:
X.shape

(500, 768)

In [23]:
y.shape

(500,)

###  Подготовка данных и обучение моделей

In [24]:
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.4, 
    stratify=y, 
    random_state=RANDOM_STATE
) 

In [25]:
def model_pipeline_randomsearch(
    X_train, 
    y_train, 
    model, 
    params, 
    data_random, 
    data_times 
):
    
    start_time = time.time() 
    
    pipeline = Pipeline([
        ('sampl', SMOTE(random_state=RANDOM_STATE)), 
        ('clf', model)
    ])
    
    random_search = RandomizedSearchCV(
        pipeline,
        params,
        scoring='f1',
        cv=5,
        n_iter=5
)
    
    random_search.fit(X_train, y_train)
    
    finish_time = time.time()
    funtion_time = finish_time - start_time
    
    data_random.append(random_search)
    data_times.append(funtion_time) 
    
    return data_random, data_times

In [26]:
def print_model_result(randoms, data_times, model_name):
    print('Модель    :', model_name)
    print('Метрика F1:', randoms[-1].best_score_)
    print(f'Время     : {data_times[-1]} секунд')
    print('Параметры :', randoms[-1].best_estimator_[-1].get_params())

In [27]:
data_random = []
data_times = []

In [28]:
params = [{
    'clf__solver': ('liblinear', 'lbfgs'),
    'clf__penalty': ('l1', 'l2'),
    'clf__C': list(range(5, 15, 1))
}]
data_random, data_times = model_pipeline_randomsearch(
    X_train, 
    y_train, 
    LogisticRegression(
        random_state=RANDOM_STATE
    ), 
    params, 
    data_random, 
    data_times 
)
print_model_result(data_random, data_times, 'LogisticRegression')

Модель    : LogisticRegression
Метрика F1: 0.33166666666666667
Время     : 1.7133798599243164 секунд
Параметры : {'C': 10, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l1', 'random_state': 42, 'solver': 'liblinear', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


In [29]:
params = [{
    'clf__learning_rate': (0.05, 1), 
    'clf__depth': (4, 6),
}]
data_random, data_times = model_pipeline_randomsearch(
    X_train, 
    y_train,
    CatBoostClassifier(iterations=50,
                       early_stopping_rounds=10,
                       random_state=RANDOM_STATE,
                       verbose=False
                      ), 
    params, 
    data_random, 
    data_times 
)
print_model_result(data_random, data_times, 'CatBoostClassifier')

Модель    : CatBoostClassifier
Метрика F1: 0.33026228673287494
Время     : 62.22422766685486 секунд
Параметры : {'iterations': 50, 'learning_rate': 1, 'depth': 4, 'verbose': False, 'random_state': 42, 'early_stopping_rounds': 10}


Выбор лучшей модели:

In [30]:
data_random_best = data_random[0]
data_times_best = data_times[0]

for i in range(0, len(data_random)):
    if data_random[i].best_score_ > data_random_best.best_score_: 
        data_random_best = data_grids[i]
        data_times_best = data_times[i]

print('Лучшее время           :', data_times_best)
print('Лучший показатель F1   :', data_random_best.best_score_)
print('Лучшая модель          :') 
print(data_random_best) 
print('Лучшие параметры модели:') 
data_random_best.get_params()

Лучшее время           : 1.7133798599243164
Лучший показатель F1   : 0.33166666666666667
Лучшая модель          :
RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('sampl', SMOTE(random_state=42)),
                                             ('clf',
                                              LogisticRegression(random_state=42))]),
                   n_iter=5,
                   param_distributions=[{'clf__C': [5, 6, 7, 8, 9, 10, 11, 12,
                                                    13, 14],
                                         'clf__penalty': ('l1', 'l2'),
                                         'clf__solver': ('liblinear',
                                                         'lbfgs')}],
                   scoring='f1')
Лучшие параметры модели:


{'cv': 5,
 'error_score': nan,
 'estimator__memory': None,
 'estimator__steps': [('sampl', SMOTE(random_state=42)),
  ('clf', LogisticRegression(random_state=42))],
 'estimator__verbose': False,
 'estimator__sampl': SMOTE(random_state=42),
 'estimator__clf': LogisticRegression(random_state=42),
 'estimator__sampl__k_neighbors': 5,
 'estimator__sampl__n_jobs': None,
 'estimator__sampl__random_state': 42,
 'estimator__sampl__sampling_strategy': 'auto',
 'estimator__clf__C': 1.0,
 'estimator__clf__class_weight': None,
 'estimator__clf__dual': False,
 'estimator__clf__fit_intercept': True,
 'estimator__clf__intercept_scaling': 1,
 'estimator__clf__l1_ratio': None,
 'estimator__clf__max_iter': 100,
 'estimator__clf__multi_class': 'auto',
 'estimator__clf__n_jobs': None,
 'estimator__clf__penalty': 'l2',
 'estimator__clf__random_state': 42,
 'estimator__clf__solver': 'lbfgs',
 'estimator__clf__tol': 0.0001,
 'estimator__clf__verbose': 0,
 'estimator__clf__warm_start': False,
 'estimator': Pi

В данный момент мы использовали только 500 случайных объектов из 159292 имеющихся в датафрейме. 
Такого количества объектов не достаточно для полноценного обучения модели предсказания классов.

#### Предсказание лучшей модели

In [31]:
start_time = time.time()
predict = data_random_best.predict(X_test)
finish_time = time.time()
funtion_time = finish_time - start_time
print('Показатель F1     :', f1_score(y_test, predict))
print(f'Время предсказания: {funtion_time} секунд')

Показатель F1     : 0.45454545454545453
Время предсказания: 0.0016160011291503906 секунд


Метрика лучшей модели на тестовой выборке равна 0.33, что не соответствует условиям задачи

###  Общий вывод

1. Заказчиком предоставлен датасет размерностью 159292 строки и 3 столбца

2. Произведено удаление неизвестного столбца 'Unnamed: 0'

3. Пропущенных значений и дубликатов не выявленно

4. Работа выполнена с итоговым датасетом размерностью 159292 строки и 2 столбца

5. Чтобы не создавать эмбеддинги слишком долго, из выборки используется только 500 случайных элементов

5. Текст преобразован в эмбеддинги с помощью BERT

6. Выполнена разбивка на тренировочную выборки с разметностью test_size=0.4

7. Выполнено обучение двух моделей LogisticRegression и CatBoostClassifier

10. Лучшей моделью на тренировочной выборке является LogisticRegression. Метрика F1 равна 0.33, что в условиях использования только 500 случайных элементов выборки не соответствет условию задачи.

11. Метрика лучшей модели на тестовой выборке равна 0.45, что также не соответствует условиям задачи